In [ ]:
!pip install llmware
!pip install faiss-cpu

Cloning into 'llmware'...
remote: Enumerating objects: 5664, done.
remote: Counting objects: 100% (1548/1548), done.
remote: Compressing objects: 100% (679/679), done.
remote: Total 5664 (delta 903), reused 1115 (delta 838), pack-reused 4116
Receiving objects: 100% (5664/5664), 532.23 MiB | 23.42 MiB/s, done.
Resolving deltas: 100% (3142/3142), done.
Updating files: 100% (468/468), done.
/content/llmware
Processing /content/llmware
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.0/670.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.4 MB/s eta 0

In [ ]:
import os
from llmware.setup import Setup
from llmware.library import Library
from llmware.prompts import Prompt, HumanInTheLoop
from llmware.retrieval import Query
from llmware.configs import LLMWareConfig

In [ ]:
LLMWareConfig().set_active_db("sqlite")

llm = "llmware/dragon-yi-6b-gguf"

In [ ]:
local_path = Setup().load_sample_files()
agreements_path = os.path.join(local_path, "AgreementsLarge")

In [ ]:
print(f"\nStarting:  Parsing 'AgreementsLarge' Folder")
msa_lib = Library().create_new_library("example6_library")
msa_lib.add_files(agreements_path)


Starting:  Parsing 'AgreementsLarge' Folder


{'docs_added': 81,
 'blocks_added': 12552,
 'images_added': 0,
 'pages_added': 1345,
 'tables_added': 10,
 'rejected_files': []}

In [ ]:
print(f"\nCompleted Parsing - now, let's look for the 'master service agreements', e.g., 'msa'")

q = Query(msa_lib)
query = '"master services agreement"'
results = q.text_search_by_page(query, page_num=1, results_only=False)


Completed Parsing - now, let's look for the 'master service agreements', e.g., 'msa'


In [ ]:
msa_docs = results["file_source"]
msa_doc_ids = results["doc_ID"]

In [ ]:
prompter = Prompt().load_model(llm)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


dragon-yi-6b-q4_k_m.gguf:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

In [ ]:
print("update: identified the following msa doc id: ", msa_doc_ids)

for i, doc_id in enumerate(msa_doc_ids):

  print("\n")
  docs = msa_docs[i]
  if os.sep in docs:
    docs = docs.split(os.sep)[-1]

  print (i+1, "Reviewing MSA - ", doc_id, docs)

  doc_filter = {"doc_ID": [doc_id]}
  termination_provisions = q.text_query_with_document_filter("termination", doc_filter)

  sources = prompter.add_source_query_results(termination_provisions)

  response = prompter.prompt_with_source("What is the notice for termination for convenience?")

  stats = prompter.evidence_comparison_stats(response)
  ev_source = prompter.evidence_check_sources(response)

  for i, resp in enumerate(response):
    print("update: llm response - ", resp)
    print("update: compare with evience- ", stats[i]["comparison_stats"])
    print("update: sources - ", ev_source[i]["source_review"])

  prompter.clear_source_materials()


update: identified the following msa doc id:  [2, 3, 6, 10, 11, 14, 24, 27, 30, 49, 50, 51, 52, 53, 55, 56, 61, 62, 65, 66]


1 Reviewing MSA -  2 Testco MSA Zeta.pdf
update: llm response -  {'llm_response': '30 days written notice.', 'prompt': 'What is the notice for termination for convenience?', 'evidence': " party  may  be   deemed  notice  of   termination  of  this  Agreement,  effective  on  the  date  of  assignment, by the other party.   Survival upon Termination or End of Term.   The   provisions  regarding  warranty,  limitation  of  liability,  confidentiality,  fees  and   expenses,  obligations  on   termination or expiration, ownership and license, and miscellaneous of this  Agreement, and \n any  work  order  or  services  description  if  the  other  party  is  in  material  breach  or   default  of any obligation that is not cured  within 15 days' notice of such breach. The TestCo agrees to pay all fees   for services performed  and  expenses  incurred  prior  to  the

In [ ]:
print("\nupdate: Prompt state saved at: ", os.path.join(LLMWareConfig.get_prompt_path(), prompter.prompt_id))

prompter.save_state()

In [ ]:
csv_output = HumanInTheLoop(prompter).export_current_interaction_to_csv()

print("\nupdate: CSV output for human review - ", csv_output)

return 0